In [4]:
# HEAVILY DRAWN FROM: 
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
import pandas as pd
import tensorflow as tf
import os
BATCH_SIZE = 2

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU:0')))

Num GPUs Available:  0


In [6]:
training_path = os.path.join("./", "final_results/", "trainment.csv")
df = pd.read_csv(training_path)
df.tail()

,ticker,polarity_title,subjectivity_title,polarity_description,subjectivity_description,Open,Close,Open-Close,target
1592,YELP,0.000,0.100000,0.182567,0.392550,29.440001,30.33,-0.889999,0
1593,YELP,0.000,0.000000,0.180139,0.416358,29.440001,30.33,-0.889999,0
1594,YELP,0.075,0.366667,0.075000,0.366667,29.440001,30.33,-0.889999,0
1595,YELP,0.000,0.000000,0.132656,0.404388,29.440001,30.33,-0.889999,0
1596,YELP,0.000,0.000000,0.099260,0.424278,29.440001,30.33,-0.889999,0


In [7]:
target = df.pop('target')

In [8]:
numeric_feature_names = [
    'polarity_title', 'subjectivity_title',
    'polarity_description', 'subjectivity_description', 
]
numeric_features = df[numeric_feature_names]
numeric_features.head()

,polarity_title,subjectivity_title,polarity_description,subjectivity_description
0,0.0,0.0,0.254308,0.624943
1,-0.5,0.7,0.131250,0.479861
2,0.0,0.0,0.151176,0.502663
3,0.0,0.0,0.098940,0.376537
4,0.5,0.5,0.100000,0.600000


In [9]:
tf.convert_to_tensor(numeric_features)

2023-05-19 11:04:02.180933: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<tf.Tensor: shape=(1597, 4), dtype=float64, numpy=
array([[ 0.        ,  0.        ,  0.25430839,  0.62494331],
       [-0.5       ,  0.7       ,  0.13125   ,  0.47986111],
       [ 0.        ,  0.        ,  0.15117624,  0.50266291],
       ...,
       [ 0.075     ,  0.36666667,  0.075     ,  0.36666667],
       [ 0.        ,  0.        ,  0.1326564 ,  0.40438842],
       [ 0.        ,  0.        ,  0.09926046,  0.4242785 ]])>

In [10]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

In [11]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_basic_model()
model.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
799/799 [==============================] - 5s 5ms/step - loss: 0.6969 - accuracy: 0.4884
Epoch 2/15
799/799 [==============================] - 4s 5ms/step - loss: 0.6906 - accuracy: 0.4872
Epoch 3/15
799/799 [==============================] - 3s 4ms/step - loss: 0.6891 - accuracy: 0.4890
Epoch 4/15
799/799 [==============================] - 4s 4ms/step - loss: 0.6887 - accuracy: 0.4997
Epoch 5/15
799/799 [==============================] - 4s 5ms/step - loss: 0.6875 - accuracy: 0.5022
Epoch 6/15
799/799 [==============================] - 4s 5ms/step - loss: 0.6868 - accuracy: 0.4984
Epoch 7/15
799/799 [==============================] - 4s 5ms/step - loss: 0.6862 - accuracy: 0.5078
Epoch 8/15
799/799 [==============================] - 6s 7ms/step - loss: 0.6858 - accuracy: 0.5110
Epoch 9/15
799/799 [==============================] - 3s 4ms/step - loss: 0.6851 - accuracy: 0.5141
Epoch 10/15
799/799 [==============================] - 5s 6ms/step - loss: 0.6846 - accuracy: 0.5147

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 4)                9         
 n)                                                              
                                                                 
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 180
Trainable params: 171
Non-trainable params: 9
_________________________________________________________________


In [15]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets
